In [1]:
import sys
sys.path.append('..')
from osp import *

In [7]:
non_content_words = get_non_content_words()
content_words = get_top_words(len(non_content_words))

In [8]:
top_words = get_top_words(1000)

In [11]:
# 'hence' in top_words

In [12]:
data_top_words = get_counts_wordset(top_words)

100%|██████████| 58126/58126 [05:36<00:00, 172.65it/s]


In [13]:
data_content_words = get_counts_wordset(content_words)
data_non_content_words = get_counts_wordset(non_content_words)
data_non_content_words

,the,of,more,an,than,but,for,him,as,his,...,somewheres,skywards,meself,yuk,northwestward,yippee,northeastward,southeastward,some-where,your-self
id,,,,,,,,,,,,,,,,,,,,,
phil/10.2307/2375809,0.165778,0.109275,0.004264,0.010661,0.006397,0.018124,0.022388,0.011194,0.017058,0.031983,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
phil/10.2307/45238188,0.103907,0.085619,0.004988,0.013023,0.007204,0.012746,0.022721,0.000554,0.016902,0.004156,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
phil/10.2307/20115538,0.142105,0.105263,0.006767,0.012782,0.000752,0.009774,0.023308,0.000000,0.014286,0.003008,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
phil/10.2307/40380502,0.118147,0.094773,0.011900,0.013175,0.009350,0.010625,0.026774,0.000000,0.028899,0.000425,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
phil/10.2307/24659549,0.155544,0.123432,0.001505,0.016056,0.006021,0.009032,0.014551,0.001505,0.021074,0.002007,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lit/2872592,0.148089,0.101831,0.007388,0.009958,0.005461,0.010922,0.012207,0.005461,0.018953,0.016383,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
phil/10.2307/42964393,0.040230,0.011494,0.000000,0.034483,0.005747,0.005747,0.000000,0.000000,0.011494,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
phil/10.2307/27293693,0.146286,0.094857,0.006286,0.022286,0.000571,0.004000,0.017714,0.000000,0.013143,0.001714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
import numpy as np

def mdw_data(data, target_col='_target'):
    # data_y = data[target_col]
    # data_X = data.drop(columns=[target_col]).fillna(0)
    # return data_X, data_y
    tval = list(sorted(data[target_col].unique()))[0]
    data_yes = data.query(f'{target_col}==@tval').drop(columns=[target_col])
    data_no = data.query(f'{target_col}!=@tval').drop(columns=[target_col])
    avg_yes = data_yes.mean()
    avg_no = data_no.mean()
    out = avg_yes / avg_no
    out = out[out != 0].replace([np.inf, -np.inf], np.nan).dropna().sort_values(ascending=False)
    return out.apply(np.log10)
    
    
    

In [60]:
o = []
for wordsetname,wordset in [('content',data_content_words),('non_content',data_non_content_words),('top',data_top_words)]:      
    odf=pd.DataFrame()
    for g,gfx in iter_pairwise_samples(wordset,'century_discipline'):
        gstr = ' vs. '.join(g)
        odf[gstr] = mdw_data(gfx)
    o.append(odf.assign(wordset=wordsetname))
odf = pd.concat(o).rename_axis('word')
odf

,C20 Literature vs. C20 Philosophy,C20 Literature vs. C21 Literature,C20 Literature vs. C21 Philosophy,C20 Philosophy vs. C21 Literature,C20 Philosophy vs. C21 Philosophy,C21 Literature vs. C21 Philosophy,wordset
word,,,,,,,
literary,1.324534,-0.270731,1.888405,-1.618839,0.563213,2.158727,content
text,1.041994,-0.002821,1.042693,-1.050236,-0.000035,1.049115,content
plays,1.021969,0.265010,0.718745,-0.764552,-0.303155,0.461819,content
story,0.939944,0.052079,0.712738,-0.884715,-0.227502,0.667905,content
english,0.928814,0.013349,1.113952,-0.928270,0.185830,1.101119,content
...,...,...,...,...,...,...,...
properties,-1.098360,-0.082145,-1.504365,1.009241,-0.405599,-1.415287,top
proposition,-1.186901,0.095324,-1.231980,1.295911,-0.045784,-1.339709,top
empirical,-1.216123,-0.390586,-1.179401,0.826074,0.037135,-0.794569,top


In [61]:
odf.to_csv('../data/mdw_data.csv')

In [62]:
topdf = odf.query('wordset=="top"').mean(numeric_only=True, axis=1).apply(abs).sort_values(ascending=False)
dict(topdf.head(100).round(2).astype(float).to_dict())

{'poem': 0.72,
 'poet': 0.72,
 'causal': 0.67,
 'literary': 0.66,
 'professor': 0.64,
 'press': 0.61,
 'poetry': 0.59,
 'properties': 0.58,
 'writers': 0.56,
 'beliefs': 0.5,
 'passages': 0.49,
 'wrote': 0.48,
 'oxford': 0.47,
 'conceptual': 0.47,
 'french': 0.46,
 'empirical': 0.45,
 'scene': 0.44,
 'relevant': 0.44,
 'love': 0.43,
 'reasoning': 0.42,
 'propositions': 0.41,
 'cambridge': 0.41,
 'century': 0.41,
 'proposition': 0.4,
 'logic': 0.4,
 'passage': 0.39,
 'argue': 0.39,
 'written': 0.39,
 'author': 0.39,
 'great': 0.39,
 'style': 0.39,
 'english': 0.39,
 'characters': 0.38,
 'felt': 0.38,
 'belief': 0.37,
 'phrase': 0.37,
 'focus': 0.36,
 'probably': 0.36,
 'himself': 0.36,
 'property': 0.36,
 'theories': 0.36,
 'model': 0.36,
 'became': 0.36,
 'years': 0.35,
 'notes': 0.35,
 'gave': 0.35,
 'rational': 0.35,
 'young': 0.35,
 'followed': 0.35,
 'objects': 0.34,
 'upon': 0.34,
 'text': 0.34,
 'concepts': 0.34,
 'whom': 0.33,
 'lines': 0.33,
 'hardly': 0.33,
 'considerable': 0.

In [63]:
PATH_MDW_DATA = os.path.join(PATH_DATA, 'mdw_data.csv')

def load_mdw_data(path=PATH_MDW_DATA):
    return pd.read_csv(path).set_index('word')

def get_mdw_words(path=PATH_MDW_DATA, wordset="top", absval=True):
    s = load_mdw_data(path).query(f'wordset=="{wordset}"').mean(numeric_only=True, axis=1)
    if absval:
        s = s.apply(abs)
    return s.sort_values(ascending=False)

def get_top_mdw_words(n=100,path=PATH_MDW_DATA, wordset="top", absval=True):
    s = get_mdw_words(path, wordset=wordset, absval=absval)
    return s.head(n).index.tolist()

In [67]:
top_words = get_top_mdw_words(wordset='top',n=None)
len(top_words)

1000

In [75]:
PATH_NON_CONTENT_WORDS = os.path.join(PATH_DATA, 'non_content_words.xlsx')

def load_non_content_words(path=PATH_NON_CONTENT_WORDS):
    return pd.read_excel(path).set_index('word')

def get_non_content_words(path=PATH_NON_CONTENT_WORDS):
    df = load_non_content_words(path)
    return df[df.incl.str.lower().str.strip() == "y"].index.tolist()

In [78]:
non_content_words = get_non_content_words()

In [79]:
top_non_content_words = [w for w in top_words if w in non_content_words]
top_content_words = [w for w in top_words if w not in non_content_words]
len(top_non_content_words), len(top_content_words)





(18, 982)

In [82]:
len(non_content_words)

110

In [56]:
load_mdw_data()

,C20 Literature vs. C20 Philosophy,C20 Literature vs. C21 Literature,C20 Literature vs. C21 Philosophy,C20 Philosophy vs. C21 Literature,C20 Philosophy vs. C21 Philosophy,C21 Literature vs. C21 Philosophy,wordset
word,,,,,,,
literary,1.378871,-0.266052,1.864721,-1.638900,0.485481,2.127277,content
plays,1.012965,0.266691,0.715448,-0.747853,-0.296548,0.450843,content
text,0.967807,-0.026908,1.067709,-0.985712,0.100194,1.084603,content
story,0.946494,0.047590,0.697338,-0.894148,-0.249875,0.650589,content
oxford,0.916910,0.054424,-0.584958,-0.853834,-1.502920,-0.636200,content
...,...,...,...,...,...,...,...
properties,-1.060901,-0.000975,-1.450037,1.037328,-0.389115,-1.433253,top
proposition,-1.114901,0.160082,-1.185168,1.266622,-0.070267,-1.329280,top
empirical,-1.194554,-0.361674,-1.166895,0.824097,0.027723,-0.801571,top


In [57]:
get_top_mdw_words(absval=False)

word
poem          0.746129
professor     0.711709
poet          0.701185
literary      0.679855
poetry        0.632437
                ...   
oxford       -0.496519
beliefs      -0.503656
properties   -0.549492
press        -0.661692
causal       -0.691850
Length: 1000, dtype: float64